In [12]:
## imports

import numpy as np
import itertools
from collections import namedtuple
import os
import random
from random import shuffle
from collections import namedtuple

import fonctions

Combo = namedtuple('Combo', 'x solution')

In [13]:
## all functions
def read_data(filename):
    array = np.fromfile(filename, dtype=int, sep=' ')
    dim = array[0]
    obj = []
    for i in range(4):
        obj.append(array[1+i*dim**2:1+(i+1)*dim**2].reshape((dim, dim)))
    return dim, obj


def init(dim, n=1):
    init = list()
    for i in range(n):
        tmp = [i for i in range(dim)]
        shuffle(tmp)
        init.append(tmp)
    return init

def get_solution(x, objs):
    """ je considère qu'on ne me donne que un vecteur de solution , et pas un vecteur de vecteur de solution."""
    obj_to_return = []
    for obj in objs:
        calcul = 0
        for i in range(len(x)):
            calcul += obj[i][x[i]]
        obj_to_return.append(calcul)
    return obj_to_return


def findNeighbours(X, nbNeighbour):
  neighbours = []
  for i in range(0, nbNeighbour):
    newNeighbour = X.copy()
    while True:
      row1 = int(round(random.random()*len(X))) % len(X)
      row2 = int(round(random.random()*len(X))) % len(X)
      tmp = newNeighbour[row1]
      newNeighbour[row1] = newNeighbour[row2]
      newNeighbour[row2] = tmp
      
      found = False
      for nb in neighbours:
        doublon = True
        for j in range(0, len(X)):
          if nb[j] != newNeighbour[j]:
            doublon = False
        if doublon == True:
          found = True

      if not found:
        neighbours.append(newNeighbour)
        break
  return neighbours

def findNeighboursA(X, nbNeighbour,nPermut):
    neighbours = []
    for i in range(0, nbNeighbour):
        newNeighbour = X.copy()
    
        while True:
            
            permut_rows = random.sample(X,nPermut)
                                        
            temps = [newNeighbour[row] for row in permut_rows]
            
            for tmp,row in zip(reversed(temps),permut_rows) :   
                newNeighbour[row] = tmp

            found = False
            for nb in neighbours:
                doublon = True
                for j in range(0, len(X)):
                    if nb[j] != newNeighbour[j]:
                        doublon = False
                if doublon == True:
                    found = True

            if not found:
                neighbours.append(newNeighbour)
                break

    return neighbours

def get_all_neighbour(x):
    x = np.array(x)
    l = len(x)
    permutations = []
    for i in range(l):
        for j in range(i+1, l):
            permutations.append([i, j])
    all_possible_permutations = []
    for permut in permutations:
        tmp = x.copy()
        tmp[permut] = x[permut[::-1]]
        all_possible_permutations.append(tmp.tolist())
    return all_possible_permutations


def x_and_sol_to_named_tuple(x_to_convert,objs) :
    
    """ Prend en entrée une liste de vecteurs X et 
    les associe à leurs solutions dans un named tuple 
    renvoit une liste de named_tuples """ 
    
    if (type(x_to_convert[0]) == list) :
        
        tuple_list = []
        for x in x_to_convert:
            #print(x)
            x_tuple = Combo(x,get_solution(x,objs))
            tuple_list.append(x_tuple)

        return tuple_list
    
    else :
        
        return [Combo(x_to_convert,get_solution(x_to_convert,objs))]
## TODO CHANGED HERE    

    
def compare_two_points(tuple1,tuple2,) :
    
    """ return an integer 0, 1 , 2 to determine order of dominance
    0 : no dominance
    1 : the first element gets dominated
    2 : the second element gets dominated """
    
    #print("first tuple", tuple1)
    #print("second tupel",tuple2)
    elem_to_del = 0
    
    solutions1 = tuple1.solution 
    solutions2 = tuple2.solution 
    
    comp_list = [-1 if obj1>obj2 else 0 if obj1==obj2 else 1 for obj1,obj2 in zip(solutions1,solutions2)]
    
    comp_set = set(comp_list)
    #print(comp_set)
    
    if {0} == comp_set :
        #print('on ne supprime rien, les obj sont egaux :')
        elem_to_del = 0 
    
    elif all(x in [0,1] for x in comp_set) | ({1} == comp_set) :
        #print('on delete le 2e element : ' , tuple2)
        elem_to_del = 2
    
    elif (all(x in [0,-1] for x in comp_set)) | ({-1} == comp_set) :
        #print('on delete le 1er element :', tuple1)
        elem_to_del = 1
        
    else :
        #print('On ne supprime rien ')
        elem_to_del = 0
    
    return elem_to_del

def compare_and_delete(tuple_group) :
    
    """Compares each tuple to all the elements in the list,
    deletes the dominated ones,
    as well as the duplicates """

    dominated = []
    unique_domi = []
    unique_tuple_group = []
    tuple_group_copy = tuple_group.copy()
    
    ## creates a list of dominated tuples
    for (tuple1,tuple2) in itertools.combinations(tuple_group_copy,2):
             
            
            who_to_del = compare_two_points(tuple1,tuple2,)
         
            if who_to_del == 1 :
                dominated.append(tuple1)
            if who_to_del == 2 :
                dominated.append(tuple2)
                
    
    ## creates unique list of dominated tuples
    unique = [unique_domi.append(x) for x in dominated if x not in unique_domi]         
    #print('the dominated vector' , unique_domi)
    
    ## delete all the dominated tuples   
    tuple_group_copy[:] =  [n for n in tuple_group_copy if n not in unique_domi]
    
    ### delete all the duplicates in the tuple list  
    unique2 = [unique_tuple_group.append(x) for x in tuple_group_copy if x not in unique_tuple_group]
         
    return unique_tuple_group
    


def add_and_update_archive(neighbour_tuples, archive_tuple) :
    
    """ pour chaque nouvel élément potentiel, comparer cet élément à tout l'archive 
    Si l'élément domine un élément de l'archive, supprimer ce dernier et ajouter le nouvel élément
    Si l'élément se fait dominer au moins une fois, supprimer l'élement 
    Sinon, ajouter à l'archive."""
    
    who_to_del = 0 
    neigh_is_worse = False
    neighs_to_add = []
    unique_archive = []
   
    archive_tuple_copy = archive_tuple.copy()
    
    for neighbour in neighbour_tuples :
        
        
        
        for arch in reversed(archive_tuple_copy) : 
            
            #print('treating another arch')
            
                   
            who_to_del = compare_two_points(neighbour,arch)
            
            if who_to_del == 2 :
                
                #print("on va del qqun dans l'arch : ", arch)
                #print("celui qui l'a éliminé ", neighbour)
                archive_tuple_copy[:] = [x for x in archive_tuple_copy if x != arch]
            
            if who_to_del == 1 :
                #print(' issue : neighbour is worse than someone in the archive')
                neigh_is_worse = True
                break
                
        if neigh_is_worse == False :
            neighs_to_add.append(neighbour)
            
        neigh_is_worse = False
        
                
            #print('after boucle if \n')   
            #print(archive_tuple_copy)
    
    #print('neight to add : \n' ,neighs_to_add)
    archive_tuple_copy.extend(neighs_to_add)
    #print('full archive \n: ' , archive_tuple_copy)
    
    unique = [unique_archive.append(x) for x in archive_tuple_copy if x not in unique_archive]
    
    #print('type unique_arch ' , type(unique_archive))
    
    return unique_archive

In [14]:
### gauthier


def update_iterations_count(change,iterations_count):
  if change==False:
    iterations_count+=1
    #print(iterations_count)
  else:
    iterations_count=0
    #print(iterations_count)
  return iterations_count


def stop_condition(iterations_count,limite):
  if iterations_count > limite:
    return 0
  else:
    return 1

In [15]:
def compare_old_and_new_archive(archive_old,archive_new) :
    
    equivalent_found = 0
    archive_changed = True;

    for new in archive_new :   
    
        for old in archive_old :    

            #print(old[0].x)
            #print(type(old))
            #print(new.x)

            if old.x == new.x :

                equivalent_found += 1
                continue;
    #print("number of equivalences found : ", equivalent_found)
    if (equivalent_found == len(archive_new) & len(archive_old) == len(archive_new)):
        archive_changed = False;

    #print('archive changed : '  , archive_changed)  
    

    return archive_changed 


# THIS IS THE MAIN

In [16]:
# nombre de points à l'initialisation
n_init = 100
## nombre d'objectifs
n_obj = 2
## nombre de voisins par voisinage
#n_neighbour = 30

## nombre de "chromosomes" permutés d'un voisin
#nPermut = 3

# l'archive a été modifiée?
archive_changed = False

# nombre d'itérations
iterations_count=0

#n_max d'itérations sans changements
limite = 50

## lecture de fichier
filepath = 'Data/'
dim, obj = read_data(filepath+'LAP-15.dat')


In [17]:
## initialisation de l'archive
ini = init(dim, n_init)
print(f"""
        dim : {dim} 
        obj : {obj}
        init : {init}
    """)


obj = obj[0:n_obj]


        dim : 15 
        obj : [array([[ 1,  6,  5,  2,  5, 10,  3,  6,  7,  8,  9,  2,  7,  2,  9],
       [ 2,  7, 10,  3, 10,  3,  6,  3,  2,  9,  6,  5,  2,  7,  6],
       [ 3,  2,  3,  6,  1,  6,  3, 10,  5,  6,  7, 10,  5,  4,  5],
       [ 2,  3,  6,  5,  8,  3,  8,  7,  8,  9,  6,  3,  4,  1,  6],
       [ 5,  6,  7,  4,  3, 10,  7,  2,  5,  4,  7,  2,  9, 10,  5],
       [ 4,  1,  2,  9, 10,  9,  4,  7,  2,  3,  8,  3,  6,  3,  6],
       [ 5,  4,  9,  6,  3,  6,  9, 10,  7,  6,  9,  6,  7, 10,  3],
       [ 2,  7, 10,  1,  8,  3,  6,  9,  6,  1,  8,  3, 10,  1, 10],
       [ 7,  6,  1,  8,  9,  8,  7,  4,  1,  6,  3,  2,  1,  8,  3],
       [ 8,  9,  2,  1, 10,  3,  4,  7,  4,  9,  4,  9,  2,  3,  4],
       [ 5,  2,  5,  4,  1,  4,  9,  2,  1,  2,  1, 10,  1,  6,  3],
       [10,  9,  8,  1,  6,  1,  6,  7,  6,  9,  6,  3,  4,  9,  8],
       [ 9,  4,  5,  4,  5,  6,  9,  4,  3,  8,  9, 10,  1,  6,  1],
       [ 2,  7,  2,  7,  6,  1,  6,  5,  6,  1,  6,  5,  6,  1,  8],


In [18]:
## obtention de tuples + cleanage archive
tupled_ini = x_and_sol_to_named_tuple(ini,obj)
archive = compare_and_delete(tupled_ini)

In [19]:
i = 0 

while (i<10000000) :
    
    #print('iteration #',i)
    
    ## select a random element from the archive
    original_neigh = random.choice(archive)
    #print(original_neigh)

    ## calcul de voisin d'un point de l'archive
    neighbourhood = get_all_neighbour(original_neigh.x)
    #neighbourhood = findNeighboursA(original_neigh.x, nbNeighbour,nPermut)

    ## calcul des solutions des voisins
    tupled_neighbourhood = x_and_sol_to_named_tuple(neighbourhood,obj)

    ## ajouter l'original au voisinage
    #tupled_neighbourhood.append(Combo(original_neigh.x,original_neigh.solution))
    tupled_neighbourhood.extend([original_neigh])

    ## Nettoyage du voisinage
    clean_neighbourhood = compare_and_delete(tupled_neighbourhood)

    ## ajouter les voisins à l'archive
    new_archive= add_and_update_archive(clean_neighbourhood,archive)

    ## comparer si l'ancien archive et la nouvelle sont identiques
    archive_changed = compare_old_and_new_archive(archive ,new_archive)
    
    ## nouvelle archive = l'ancienne
    archive = new_archive
    
    if (i%1000 == 0) :
        print('iteration #',i)
        print("LA nvlle archive : ", archive)

    ## condition d'arret
    iterations_count = update_iterations_count(archive_changed, iterations_count)
    test = stop_condition(iterations_count,limite)  
    
    i += 1
    #print('iteration #',i)
    #print("LA nvlle archive : ", archive)
    #print("iterations_count", iterations_count,)
    
    
    if test == False:
        print("#iteration :", i)
        print("test", test , "iterations_count", iterations_count, "limite", limite)
        print("FINAL ARCHIVE : ", archive)
        break


iteration # 0
LA nvlle archive :  [Combo(x=[11, 14, 5, 8, 9, 4, 10, 12, 2, 1, 13, 6, 3, 7, 0], solution=[91, 84]), Combo(x=[0, 5, 12, 13, 9, 7, 10, 6, 8, 11, 1, 4, 14, 2, 3], solution=[59, 112]), Combo(x=[6, 0, 12, 13, 9, 7, 10, 5, 8, 11, 1, 4, 14, 2, 3], solution=[57, 116]), Combo(x=[6, 5, 12, 0, 9, 7, 10, 13, 8, 11, 1, 4, 14, 2, 3], solution=[57, 116]), Combo(x=[6, 5, 12, 13, 9, 7, 11, 0, 8, 10, 1, 4, 14, 2, 3], solution=[49, 144]), Combo(x=[6, 5, 12, 13, 9, 7, 4, 0, 8, 11, 1, 10, 14, 2, 3], solution=[51, 120])]
#iteration : 435
test 0 iterations_count 51 limite 50
FINAL ARCHIVE :  [Combo(x=[13, 7, 3, 0, 9, 6, 4, 5, 2, 14, 1, 11, 8, 10, 12], solution=[51, 52]), Combo(x=[13, 7, 3, 0, 9, 6, 1, 5, 2, 14, 4, 11, 8, 10, 12], solution=[51, 52]), Combo(x=[13, 7, 3, 0, 10, 6, 4, 5, 2, 14, 1, 11, 8, 9, 12], solution=[49, 56]), Combo(x=[13, 7, 3, 0, 9, 6, 4, 5, 2, 14, 12, 11, 8, 10, 1], solution=[49, 56]), Combo(x=[13, 7, 1, 0, 9, 6, 4, 5, 2, 14, 8, 11, 3, 10, 12], solution=[47, 60]), Combo(x=

### save the file numbers

In [2]:
print('nos_solutions\ trentex30_2obj.txt')
with open('nos_solutions\\trentex30_2obj.txt','w')  as file:
    
    for arch in archive :
        for x in arch.x :
            file.write(str(x))
            file.write(' ')
        file.write('\n')
        print (arch.x)
    
    for arch in archive :
        for xol in arch.solution :
            file.write(str(xol))
            file.write(' ')
        file.write('\n')
        print (arch.solution)
    

nos_solutions\ trentex30_2obj.txt


NameError: name 'archive' is not defined